In [182]:
%reset
import sqlite3
import pandas as pd
import numpy as np

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [221]:
class FootballData:
    """
    Class to get and describe the data from kaggle soccer dataset
    """
    def __init__(self, data_path):
        """
        Initialize data from kaggle to use in other methods
        """
        self.cnx = sqlite3.connect(data_path)

        self.country_df = pd.read_sql_query("SELECT * FROM Country", self.cnx)
        self.league_df = pd.read_sql_query("SELECT * FROM League", self.cnx)
        self.match_df = pd.read_sql_query("SELECT * FROM Match", self.cnx)
        self.player_df = pd.read_sql_query("SELECT * FROM Player", self.cnx)
        self.player_att_df = pd.read_sql_query("SELECT * FROM Player_Attributes", self.cnx)
        self.team_df = pd.read_sql_query("SELECT * FROM Team", self.cnx)
        self.team_att_df = pd.read_sql_query("SELECT * FROM Team_Attributes", self.cnx)
        
    def get_league_season_data(self, league, season):
        """
        Return all match data from a competition in a season
        """
        league_api_id = self.league_df.loc[(self.league_df['name'] == league)]['id'].values[0]
        
        league_matches = self.match_df.loc[(self.match_df['league_id'] == league_api_id) &  
                                           (self.match_df['season'] == season)].copy()
        
        league_matches["points_for_home"] = np.where(league_matches['home_team_goal'] > league_matches['away_team_goal'], 3,
                                             np.where(league_matches['home_team_goal'] == league_matches['away_team_goal'], 1, 0))
        league_matches["points_for_away"] = np.where(league_matches['home_team_goal'] < league_matches['away_team_goal'], 3,
                                             np.where(league_matches['home_team_goal'] == league_matches['away_team_goal'], 1, 0))
        
        
        home_games = league_matches.groupby('home_team_api_id').sum().sort_values("points_for_home")
        away_games = league_matches.groupby('away_team_api_id').sum().sort_values("points_for_away")
        
        
    
    def get_team_season_data(self, team, season):
        """
        Return all match data from a team in a particular season
        """
        # Get the data from the dataframe
        team_api_id = self.team_df.loc[(self.team_df['team_long_name'] == team)]['team_api_id'].values[0]
        team_matches = self.match_df.loc[((self.match_df['home_team_api_id'] == team_api_id) | 
                                          (self.match_df['away_team_api_id'] == team_api_id))
                                         & (self.match_df['season'] == season)].copy()
        
        # Add Home and away column to able to differentiate between games played home and games played away
        team_matches["home/away"] = np.where(team_matches['home_team_api_id'] == team_api_id, "home", "away")

        team_matches["punten"] = np.where(((team_matches['home_team_goal'] > team_matches['away_team_goal']) & (team_matches["home/away"] == "home")) |
                                          ((team_matches['home_team_goal'] < team_matches['away_team_goal']) & (team_matches["home/away"] == "away")), 3, 
                                          np.where(team_matches['home_team_goal'] == team_matches['away_team_goal'], 1, 0))
        
        return team_matches
    
    def describe_team_season_data(self, data):
        """
        Print a description of the team/season data
        """  
        home_win_perc = round(len(data["punten"].loc[(data["punten"] == 3) & (data["home/away"] == "home")]) / len(data.loc[data["home/away"] == "home"]) * 100, 2)
        home_draw_perc = round(len(data["punten"].loc[(data["punten"] == 1) & (data["home/away"] == "home")]) / len(data.loc[data["home/away"] == "home"]) * 100, 2)
        home_loss_perc = round(len(data["punten"].loc[(data["punten"] == 0) & (data["home/away"] == "home")]) / len(data.loc[data["home/away"] == "home"]) * 100, 2)
        
        away_win_perc = round(len(data["punten"].loc[(data["punten"] == 3) & (data["home/away"] == "away")]) / len(data["punten"].loc[data["home/away"] == "away"]) * 100, 2)
        away_draw_perc = round(len(data["punten"].loc[(data["punten"] == 1) & (data["home/away"] == "away")]) / len(data["punten"].loc[data["home/away"] == "away"]) * 100, 2)
        away_loss_perc = round(len(data["punten"].loc[(data["punten"] == 0) & (data["home/away"] == "away")]) / len(data["punten"].loc[data["home/away"] == "away"]) * 100, 2)
        
        
        print(f"Aantal wedstrijden: {str(len(data))} \n\
                Aantal punten: {str(data['punten'].sum())} \n\
                Gemiddelde punten per wedstrijd: {str(round(data['punten'].mean(), 2))} \n\n\
                Win thuis percentage: {str(home_win_perc)}% \n\
                Gelijk thuis percentage: {str(home_draw_perc)}% \n\
                Verlies thuis percentage: {str(home_loss_perc)}% \n\n\
                Win uit percentage: {str(away_win_perc)}% \n\
                Gelijk uit percentage: {str(away_draw_perc)}% \n\
                Verlies uit percentage: {str(away_loss_perc)}%")
        
        
        
        
        

In [222]:
fd = FootballData("database.sqlite")

In [207]:
fd.describe_team_season_data(fd.get_team_season_data("Feyenoord", "2008/2009"))

Aantal wedstrijden: 34 
                Aantal punten: 45 
                Gemiddelde punten per wedstrijd: 1.32% 

                Win thuis percentage: 47.06% 
                Gelijk thuis percentage: 29.41% 
                Verlies thuis percentage: 23.53% 

                Win uit percentage: 23.53% 
                Gelijk uit percentage: 23.53% 
                Verlies uit percentage: 52.94%


In [223]:
fd.get_league_season_data('Netherlands Eredivisie','2008/2009')

C:\Users\Mark\AppData\Local\Temp\ipykernel_13020\3302338939.py:35: FutureWarning: In a future version of pandas all arguments of Series.sort_values will be keyword-only.
  home_games = league_matches.groupby('home_team_api_id').size().sort_values("points_for_home")


ValueError: No axis named points_for_home for object type Series

In [152]:
def ranglijst(league, seizoen):
    leagueapid = league_df.loc[(league_df['name'] == league)]['id'].values[0]
    ranglijst = pd.DataFrame()
    ranglijst['Team_id']=match_df.loc[(match_df['league_id'] == leagueapid) &  (match_df['season'] == seizoen), 'home_team_api_id'].unique()
    ranglijst['Team_Name'] = pd.merge(ranglijst, team_df, left_on =ranglijst['Team_id'], right_on = team_df['team_api_id'])['team_long_name']
    match_df['home_outcome'] = np.where(match_df['home_team_goal'] > match_df['away_team_goal'], 'W', np.where(match_df['home_team_goal'] == match_df['away_team_goal'], 'D', 'L'))
    match_df['home_punten']= np.where(match_df['home_outcome'] == 'W', 3, np.where(match_df['home_outcome'] == 'D', 1, 0))
    match_df['away_outcome'] = np.where(match_df['home_team_goal'] < match_df['away_team_goal'], 'W', np.where(match_df['home_team_goal'] == match_df['away_team_goal'], 'D', 'L'))
    match_df['away_punten']= np.where(match_df['away_outcome'] == 'W', 3, np.where(match_df['away_outcome'] == 'D', 1, 0))
    match_df['home_wins']=np.where(match_df['home_outcome'] == 'W', 1,0)
    match_df['home_draws']=np.where(match_df['home_outcome'] == 'D', 1,0)
    match_df['home_loses']=np.where(match_df['home_outcome'] == 'L', 1,0)    
    match_df['away_wins']=np.where(match_df['away_outcome'] == 'W', 1,0)
    match_df['away_draws']=np.where(match_df['away_outcome'] == 'D', 1,0)
    match_df['away_loses']=np.where(match_df['away_outcome'] == 'L', 1,0)
    
    Home = match_df.groupby(['league_id','season','home_team_api_id']).aggregate({'home_punten' : 'sum', 'home_team_goal' : 'sum', 'away_team_goal' : 'sum','home_wins': 'sum', 'home_draws': 'sum','home_loses' : 'sum','away_wins': 'sum', 'away_draws': 'sum','away_loses' : 'sum'})
    home_wins=Home['home_wins']
    home_wins_totaal=pd.DataFrame(home_wins[leagueapid,seizoen])
    home_draws=Home['home_draws']
    home_draws_totaal=pd.DataFrame(home_draws[leagueapid,seizoen])
    home_loses=Home['home_loses']
    home_loses_totaal=pd.DataFrame(home_loses[leagueapid,seizoen])
    Home_punten=Home['home_punten']
    home_punten_totaal=pd.DataFrame(Home_punten[leagueapid,seizoen])
    home_goals_voor=Home['home_team_goal']
    home_goals_voor_totaal=pd.DataFrame(home_goals_voor[leagueapid,seizoen])
    home_goals_tegen=Home['away_team_goal']
    home_goals_tegen_totaal=pd.DataFrame(home_goals_tegen[leagueapid,seizoen])
    Away = match_df.groupby(['league_id','season','away_team_api_id']).aggregate({'away_punten' : 'sum', 'home_team_goal' : 'sum', 'away_team_goal' : 'sum','home_wins': 'sum', 'home_draws': 'sum','home_loses' : 'sum','away_wins': 'sum', 'away_draws': 'sum','away_loses' : 'sum'})
    away_wins=Away['away_wins']
    away_wins_totaal=pd.DataFrame(away_wins[leagueapid,seizoen])
    away_draws=Away['away_draws']
    away_draws_totaal=pd.DataFrame(away_draws[leagueapid,seizoen])
    away_loses=Away['away_loses']
    away_loses_totaal=pd.DataFrame(away_loses[leagueapid,seizoen])
    away_punten=Away['away_punten']
    away_punten_totaal=pd.DataFrame(away_punten[leagueapid,seizoen])
    away_goals_tegen=Away['home_team_goal']
    away_goals_tegen_totaal=pd.DataFrame(away_goals_tegen[leagueapid,seizoen])
    away_goals_voor=Away['away_team_goal']
    away_goals_voor_totaal=pd.DataFrame(away_goals_voor[leagueapid,seizoen])
    
    ranglijst['home_punten'] = pd.merge(ranglijst, home_punten_totaal, left_on =ranglijst['Team_id'], right_index=True)['home_punten']
    ranglijst['away_punten'] = pd.merge(ranglijst, away_punten_totaal, left_on =ranglijst['Team_id'], right_index=True)['away_punten']
    ranglijst['home_goals_voor'] = pd.merge(ranglijst, home_goals_voor_totaal, left_on =ranglijst['Team_id'], right_index=True)['home_team_goal']
    ranglijst['home_goals_tegen'] = pd.merge(ranglijst, home_goals_tegen_totaal, left_on =ranglijst['Team_id'], right_index=True)['away_team_goal']
    ranglijst['away_goals_voor'] = pd.merge(ranglijst, away_goals_voor_totaal, left_on =ranglijst['Team_id'], right_index=True)['away_team_goal']
    ranglijst['away_goals_tegen'] = pd.merge(ranglijst, away_goals_tegen_totaal, left_on =ranglijst['Team_id'], right_index=True)['home_team_goal']
    
    ranglijst['home_wins'] = pd.merge(ranglijst, home_wins_totaal, left_on =ranglijst['Team_id'], right_index=True)['home_wins']
    ranglijst['away_wins'] = pd.merge(ranglijst, away_wins_totaal, left_on =ranglijst['Team_id'], right_index=True)['away_wins']
    ranglijst['home_draws'] = pd.merge(ranglijst, home_draws_totaal, left_on =ranglijst['Team_id'], right_index=True)['home_draws']
    ranglijst['away_draws'] = pd.merge(ranglijst, away_draws_totaal, left_on =ranglijst['Team_id'], right_index=True)['away_draws']
    ranglijst['home_loses'] = pd.merge(ranglijst, home_loses_totaal, left_on =ranglijst['Team_id'], right_index=True)['home_loses']
    ranglijst['away_loses'] = pd.merge(ranglijst, away_loses_totaal, left_on =ranglijst['Team_id'], right_index=True)['away_loses']
    
    ranglijst['Totaal_Gespeeld'] = ranglijst['home_wins'] + ranglijst['home_draws'] + ranglijst['home_loses'] + ranglijst['away_wins'] + ranglijst['away_draws'] + ranglijst['away_loses']
    ranglijst['Totaal_Win'] = ranglijst['home_wins'] + ranglijst['away_wins']
    ranglijst['Totaal_Draw'] = ranglijst['home_draws'] + ranglijst['away_draws']
    ranglijst['Totaal_Loss'] = ranglijst['home_loses'] + ranglijst['away_loses']
    ranglijst['Doelpunten_Voor'] = ranglijst['home_goals_voor'] + ranglijst['away_goals_voor']
    ranglijst['Doelpunten_Tegen'] = ranglijst['home_goals_tegen'] + ranglijst['away_goals_tegen']
    ranglijst['Doelpunten_Saldo'] = ranglijst['Doelpunten_Voor'] - ranglijst['Doelpunten_Tegen']
    ranglijst['Totale_punten'] = ranglijst['home_punten'] + ranglijst['away_punten']
    
    ranglijst.drop('home_wins', inplace=True, axis=1)
    ranglijst.drop('home_draws', inplace=True, axis=1)
    ranglijst.drop('home_loses', inplace=True, axis=1)
    ranglijst.drop('away_wins', inplace=True, axis=1)
    ranglijst.drop('away_draws', inplace=True, axis=1)
    ranglijst.drop('away_loses', inplace=True, axis=1)
    ranglijst.drop('home_punten', inplace=True, axis=1)
    ranglijst.drop('away_punten', inplace=True, axis=1)
    ranglijst.drop('home_goals_voor', inplace=True, axis=1)
    ranglijst.drop('away_goals_voor', inplace=True, axis=1)
    ranglijst.drop('home_goals_tegen', inplace=True, axis=1)
    ranglijst.drop('away_goals_tegen', inplace=True, axis=1)
    ranglijst = ranglijst.sort_values(by=['Totale_punten','Doelpunten_Saldo'], ascending=False, ignore_index=True)
    ranglijst.index= ranglijst.index +1
    return ranglijst
ranglijst('Netherlands Eredivisie','2008/2009')

    
    

NameError: name 'league_df' is not defined